In [1]:
# ------- this challenge is for practice ML : KPMG ------- # 

# basic tools
import os
import glob
import math
import re

# basic tools for ML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Deep Learning
import tensorflow as tf


# import Model
# if you have "Importing plotly failed. Interactive plots will not work." ERROR,
# it is from prophet: INSTALL `pip install plotly` 
from prophet import Prophet
from sklearn.model_selection import GridSearchCV    # parameter
from sklearn.model_selection import train_test_split # train, test set

In [2]:
# load data
# if you have "UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 14: invalid start byte" ERROR,
# it's because encoding form is cp949, not pd default encoding form: utf-8
# cp949: extension version of euc-kr
# most of governmen data is follow cp949

def load_data(PATH:str, dt_cols:list, encoding_format:str='cp949') -> pd.DataFrame:
    return pd.read_csv(PATH, parse_dates=dt_cols, encoding=encoding_format)

train = load_data('./data/train.csv', ['date_time'])
test = load_data('./data/test.csv', ['date_time'])

In [3]:
train.head()

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00:00:00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01:00:00,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02:00:00,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03:00:00,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04:00:00,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0


In [4]:
# no need to fill out null data
# display(train.isnull().sum())

# should be cared: 기온, 풍속, 습도, 강수량, 일조, 비전기냉방시설운영, 태양광보유
display(test.isnull().sum())

num                0
date_time          0
기온(°C)          6720
풍속(m/s)         6720
습도(%)           6720
강수량(mm, 6시간)    8400
일조(hr, 3시간)     6720
비전기냉방설비운영       7784
태양광보유           8456
dtype: int64

In [5]:
train.describe()

,num,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
count,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000,122400.000000
mean,30.500000,2324.830866,24.251713,2.151641,80.169848,0.514989,0.213533,0.683333,0.483333
std,17.318173,2058.999326,3.407902,1.514475,15.525862,2.624505,0.370517,0.465178,0.499724
min,1.000000,0.000000,11.100000,0.000000,19.000000,0.000000,0.000000,0.000000,0.000000
25%,15.750000,1055.268000,21.800000,1.100000,70.000000,0.000000,0.000000,0.000000,0.000000
50%,30.500000,1700.352000,24.200000,1.900000,84.000000,0.000000,0.000000,1.000000,0.000000
75%,45.250000,2780.487000,26.500000,2.900000,93.000000,0.000000,0.300000,1.000000,1.000000
max,60.000000,17739.225000,36.300000,20.100000,100.000000,81.500000,1.000000,1.000000,1.000000


In [6]:
test.describe()

,num,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
count,10080.000000,3360.000000,3360.000000,3360.000000,1680.000000,3360.000000,2296.0,1624.0
mean,30.500000,27.805804,2.440327,81.963095,2.191845,0.612917,1.0,1.0
std,17.318961,2.378122,1.835905,11.900843,6.786772,0.929587,0.0,0.0
min,1.000000,22.100000,0.000000,44.000000,0.000000,0.000000,1.0,1.0
25%,15.750000,26.000000,1.100000,74.000000,0.000000,0.000000,1.0,1.0
50%,30.500000,27.500000,2.100000,83.000000,0.000000,0.000000,1.0,1.0
75%,45.250000,29.200000,3.200000,92.000000,0.900000,1.000000,1.0,1.0
max,60.000000,35.400000,22.500000,100.000000,83.500000,3.000000,1.0,1.0


#### `.describe()` overview
- 기온은 `std` 2이므로 편차가 크지는 않음. 6720개가 모두 0일 수는 없으므로 아마 미관측 값일 것 -> 채울 방법 필요
    -  풍속, 습도도 결측치 개수가 같다. 기상청 사정상 관측을 하지 못한 날으로 판단 됨
- 단, 풍속의 경우 0에서 3 사이의 값이므로 최빈값으로 채워도 되지 않을까 -> 최대한 영향 덜 미치는 값으로
    - 바람이 시원하면 냉방을 덜할테니 관계가 아예 없지는 않을텐데
- 강수량은 3분위까지 0이므로: 비가 왔느냐 아니냐로 판단하는게 나을듯하고
- 일조도 3분위까지 0이므로: 0으로 결측치를 채울 것
- 냉방 설비 운영은 0과 1인듯 한데 0이 없으므로 0으로 채울 것
- 태양광 보유도 냉방 설비 운영과 마찬가지

#### 결론
|기온|풍속|습도|강수량|일조|비전기냉방설비|태양광보유|
|-:|-:|-:|-:|-:|-:|-:|
|최빈?|기온과 동일|기온과 동일|0 또는 1로 변환|0으로 대체|0으로 대체|0으로 대체|

In [7]:
# 전처리: column name 특수문자 제거

def make_column_name_simple(df:pd.DataFrame) -> pd.DataFrame:    # y값 따로 분리?
    temp_df = df.copy()
    # df.columns = ['건물번호', '일시', '전력사용', '기온', '풍속', '습도', '강수량', '일조시간', '비전기냉방', '태양광']

    # for others
    REGEX = "[^A-Za-z0-9가-힣]"
    temp_df.columns = [re.sub(REGEX, "", col_name) for col_name in temp_df.columns]
    return temp_df

In [8]:
simple_name_train = make_column_name_simple(train)
simple_name_test = make_column_name_simple(test)

In [9]:
# 전처리: 일, 시 분리
def separate_datetime(df:pd.DataFrame, dt_col:str, parsed=True, dt_format:str='%Y-%m-%d %H:%M:%S') -> pd.DataFrame:
    # 원본 보존하기 위해 임시 df 생성
    temp_df:pd.DataFrame = df.copy()
    if not parsed:
        # 날짜 분리: Dtype datetime 으로 변경
        pd.to_datetime(temp_df[dt_col], format=dt_format)

    # 분리된 열 생성    
    temp_df['year'] = temp_df[dt_col].dt.year
    temp_df['month'] = temp_df[dt_col].dt.month
    temp_df['date'] = temp_df[dt_col].dt.date
    try:
        temp_df['hour'] = temp_df[dt_col].dt.hour
    except:
        print("datetime error: hour data doesn't exist or available")
    
    # 묶여있던 컬럼을 제거
    temp_df.drop(dt_col, axis=1, inplace=True)
    print(f"[SUCCESS] column name {dt_col} successfully dropped")
    df.head(2)

    # return
    return temp_df

In [56]:
dt_seperated_train:pd.DataFrame = separate_datetime(simple_name_train, 'datetime')
dt_seperated_test:pd.DataFrame = separate_datetime(simple_name_test, 'datetime')

[SUCCESS] column name datetime successfully dropped
[SUCCESS] column name datetime successfully dropped


In [57]:
# 전처리: 결측치 (일조, 냉방, 태양광) `.fillna(0)`

def fill_na(df:pd.DataFrame, col_names:list, fill:float or int) -> pd.DataFrame:
    temp_df = df.copy()
    for col_name in col_names:
        temp_df[col_name].fillna(fill, inplace=True)
    return temp_df

In [58]:
print(dt_seperated_train.columns)
print(dt_seperated_test.columns)

Index(['num', '전력사용량kWh', '기온C', '풍속ms', '습도', '강수량mm', '일조hr', '비전기냉방설비운영',
       '태양광보유', 'year', 'month', 'date', 'hour'],
      dtype='object')
Index(['num', '기온C', '풍속ms', '습도', '강수량mm6시간', '일조hr3시간', '비전기냉방설비운영', '태양광보유',
       'year', 'month', 'date', 'hour'],
      dtype='object')


In [63]:
fill_0_train = fill_na(df=dt_seperated_train, col_names=['일조hr', '비전기냉방설비운영', '태양광보유', '강수량mm'], fill=0)
fill_0_test = fill_na(df=dt_seperated_test, col_names=['일조hr3시간', '비전기냉방설비운영', '태양광보유', '강수량mm6시간'], fill=0)

> 강수량은 어떻게 채울 것 인가?

```py
# 전처리를 위한 시각화
dt_seperated_train[['강수량mm']].hist()
dt_seperated_test[['강수량mm6시간']].hist()
```

```
=====  train  =====
강수량
0 : 103473
1 : 18927
null : 0
```

1. 0의 값이 압도적으로 많으므로 <s>0을 제외한 train의 강수량 절반을 기준으로?</s><br>
2. 비가 오지 않을 때 / 조금 올 때 / 많이 올 때 세가지로 구분한다.
    - 대충 train도 개수가 맞아야 학습이 될텐데
    - 당연히 말이 되기도 해야하고
3. (재설정) 기준: 기상 정보이므로 기상청 기준

| 시간 당 강수량 | 강수 표현 | <font color="black">해결</font> |
|-:|-:|-:|
| 0 mm | 비가 내리지 않음 | <font color="green">0</font> |
| 3 mm 미만 | 약한 비, 옷이 젖는 것은 신경쓰지 않을 정도 | <font color="yellow">약한 비</font> |
| 3 mm 이상 15 mm 미만 | 보통 비, 나뭇잎에 물방울이 맺히는 정도 | <font color="yellow">약한 비</font> |
| 15 mm 이상 30 mm 미만 | 강한 비, 큰 물 웅덩이가 고임 | <font color="orange">비</font> |
| 30 mm 이상 | 매우 강한 비, 우산이나 우비가 소용 없는 호우 주의보 수준 | <font color="orange">비</font> |

```
dt_seperated_train[dt_seperated_train['강수량mm'] > 0]['강수량mm'].describe()
count    18927.000000
mean         3.330412
std          5.930390
min          0.100000
25%          0.300000
50%          1.000000
75%          3.500000
max         81.500000
Name: 강수량mm, dtype: float64
```

In [69]:
def binding_rain(df:pd.DataFrame, col_name:str) -> pd.DataFrame:
    temp_df = df.copy()
    # ~ 약한 비
    temp_df[(temp_df[col_name] > 0) & (temp_df[col_name] < 15)][col_name] = 1
    # 보통 비 ~ 
    temp_df[temp_df[col_name] >= 15][col_name] = 2
    return temp_df

In [75]:
binding_rain_train = binding_rain(fill_0_train, '강수량mm')
binding_rain_test = binding_rain(fill_0_test, '강수량mm6시간')

# train
# 0.0    103473
# 1.0     17982
# 2.0       945

# test
# 0.0    9444
# 1.0     577
# 2.0      59

/var/folders/rl/45hg875x1rd0tqbt0wx9whm80000gn/T/ipykernel_44103/319134711.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[(temp_df[col_name] > 0) & (temp_df[col_name] < 15)][col_name] = 1
/var/folders/rl/45hg875x1rd0tqbt0wx9whm80000gn/T/ipykernel_44103/319134711.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[temp_df[col_name] >= 15][col_name] = 2


In [80]:
# 결측치 채움
binding_rain_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10080 entries, 0 to 10079
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   num        10080 non-null  int64  
 1   기온C        3360 non-null   float64
 2   풍속ms       3360 non-null   float64
 3   습도         3360 non-null   float64
 4   강수량mm6시간   10080 non-null  float64
 5   일조hr3시간    10080 non-null  float64
 6   비전기냉방설비운영  10080 non-null  float64
 7   태양광보유      10080 non-null  float64
 8   year       10080 non-null  int64  
 9   month      10080 non-null  int64  
 10  date       10080 non-null  object 
 11  hour       10080 non-null  int64  
dtypes: float64(7), int64(4), object(1)
memory usage: 945.1+ KB


In [84]:
# 전처리: 기온 풍속 습도 모두 일단 0
filled_train = fill_na(df=dt_seperated_train, col_names=['기온C', '풍속ms', '습도', '비전기냉방설비운영', '태양광보유'], fill=0)
filled_test = fill_na(df=dt_seperated_test, col_names=['기온C', '풍속ms', '습도', '비전기냉방설비운영', '태양광보유'], fill=0)

In [103]:
train.corr().sort_values(by='전력사용량(kWh)')[['전력사용량(kWh)']]

,전력사용량(kWh)
습도(%),-0.077199
num,-0.063193
강수량(mm),-0.006028
풍속(m/s),0.109965
일조(hr),0.112131
태양광보유,0.116344
비전기냉방설비운영,0.150799
기온(°C),0.158361
전력사용량(kWh),1.000000


In [104]:
# 상관관계가 심각한데...?

filled_train.corr().sort_values(by='전력사용량kWh')[['전력사용량kWh']]

,전력사용량kWh
습도,-0.077199
num,-0.063193
강수량mm,-0.006028
month,0.052146
hour,0.091472
풍속ms,0.109965
일조hr,0.112131
태양광보유,0.116344
비전기냉방설비운영,0.150799
기온C,0.158361


In [ ]:
## 파생변수 조작

# TODO 1: 월 상반기, 하반기로 분리 또는 변형하여 월 데이터에 합산 (상:.0 하:.5)
def is_early_month(df:pd.DataFrame, date:int):
    temp_df = df.copy()

    # drop_date
    return temp_df

# 날짜 파생변수: 계절, 기준은 일단 임의로 설정
def make_season(df:pd.DataFrame, month:int):
    temp_df = df.copy()
    temp_df[(temp_df['month'] >= 4) & (temp_df['month'] < 6)]['season'] = 'spring'
    temp_df[(temp_df['month'] >= 6) & (temp_df['month'] < 9)]['season'] = 'summer'
    temp_df[(temp_df['month'] >= 9) & (temp_df['month'] < 12)]['season'] = 'autumn'
    temp_df[(temp_df['month'] <= 12) & (temp_df['month'] < 4)]['season'] = 'winter'
    return temp_df

# TODO 2: DI 불쾌지수 discomfort index
# DI 9/5Ta-0.55(1-RH)(9/5Ta-26)+32
# Ta : 건구온도 (℃)
# RH : 상대습도 (소수단위)
def is_comfort(df:pd.DataFrame, ta_column:str, rh_column:str):    # ta: 기온, rh: 습도
    temp_df = df.copy()
    if temp_df['season'] != 'summer':
        return temp_df[['불쾌지수']] = 0
    ta, rh = temp_df[[ta_column]], temp_df[[rh_column]]
    temp_df[['불쾌지수']] = 9/5 * ta - 0.55 * (1-rh) * (9/5 * ta - 26) + 32
    # ~ 20 : 0
    # 20 ~ 25 : 1
    # 25 ~ : 2
    temp_df[(temp_df['불쾌지수'] < 20)]['discomfort'] = 0
    temp_df[(temp_df['불쾌지수'] >= 20) & (temp_df['불쾌지수'] < 25)]['discomfort'] = 1
    temp_df[(temp_df['불쾌지수'] >= 25)]['discomfort'] = 2    
    # TODO: drop 불쾌지수
    return temp_df
